In [1]:
%load_ext dotenv
%dotenv

## Import Dependencies

In [2]:
import os

# Imports for IBM Watson's Speech to Text Transcriber
from ibm_watson import SpeechToTextV1
from ibm_watson.websocket import RecognizeCallback, AudioSource 
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

# Imports for VADER - Text Sentiment Analysis
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# nltk
import nltk
from nltk import tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Danny\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Setup STT Service

In [3]:
api_key = os.getenv("api_key")
url = os.getenv("url")

In [4]:
# Setup Service
authenticator = IAMAuthenticator(api_key)
stt = SpeechToTextV1(authenticator=authenticator)
stt.set_service_url(url)

## Open Audio Source and Transcribe

In [6]:
# Perform transcription
with open('assets/audio_files/Alice_in_Wonderland_clip_1.mp3', 'rb') as f:
    res = stt.recognize(audio=f, content_type='audio/mp3', model='en-US_NarrowbandModel', continuous=True).get_result()

In [9]:
text = ""
for i in range(len(res['results'])):
    text += res['results'][i]['alternatives'][0]['transcript'][0:-1] + ". "

In [10]:
# Print each transcribed line with its confidence
for i in range(len(res['results'])):
    print(f"Transcript: {res['results'][i]['alternatives'][0]['transcript'][0:-1]}\n")
    print(f"Confidence: {res['results'][i]['alternatives'][0]['confidence']}\n")

Transcript: Alice in Wonderland by Lewis Carroll chapter one down the rabbit hole

Confidence: 0.91

Transcript: Alice was beginning to get very tired of sitting by her sister on the bank and of having nothing to do once or twice she had peeped into the book her sister was reading but it had no pictures or conversations in it and what is the use of a book thought Alice without pictures our conversation so she was considering in her own mind as well as she could

Confidence: 0.95



In [14]:
# The transcribed text vs the actual text
print(f"Transcribed text\n\n{text}")

Transcribed text

Alice in Wonderland by Lewis Carroll chapter one down the rabbit hole. Alice was beginning to get very tired of sitting by her sister on the bank and of having nothing to do once or twice she had peeped into the book her sister was reading but it had no pictures or conversations in it and what is the use of a book thought Alice without pictures our conversation so she was considering in her own mind as well as she could. 


##### Actual Text
Alice in Wonderland by Lewis Carroll 
Chapter One 
Down The Rabbit Hole

Alice was beginning to get very tired of sitting by her sister on the bank, and of having nothing to do: once or twice she had peeped into the book her sister was reading, but it had no pictures or conversations in it, 'and what is the use of a book,' thought Alice 'without pictures or conversation?'

So she was considering in her own mind, as well as she could

## Sentiment Analysis on the transcribed text

In [15]:
# Initialize the Sentiment Intensity Analyzer
analyzer = SentimentIntensityAnalyzer()

In [16]:
# Create a list of sentences from the transcript and initialize the sentiment dictionary
sentence_list = tokenize.sent_tokenize(text)
sentiments = {'compound': 0.0, 'neg': 0.0, 'neu': 0.0, 'pos': 0.0}

In [17]:
# Analyze sentiments for each sentence in the list
for sentence in sentence_list:
    vs = analyzer.polarity_scores(sentence)
    sentiments['compound'] += vs['compound']
    sentiments['neg'] += vs['neg']
    sentiments['neu'] += vs['neu']
    sentiments['pos'] += vs['pos']
    print(f'Sentence: {sentence}\nSentiments: {sentiments}\n\n')

Sentence: Alice in Wonderland by Lewis Carroll chapter one down the rabbit hole.
Sentiments: {'compound': 0.0, 'neg': 0.0, 'neu': 1.0, 'pos': 0.0}


Sentence: Alice was beginning to get very tired of sitting by her sister on the bank and of having nothing to do once or twice she had peeped into the book her sister was reading but it had no pictures or conversations in it and what is the use of a book thought Alice without pictures our conversation so she was considering in her own mind as well as she could.
Sentiments: {'compound': -0.3064, 'neg': 0.066, 'neu': 1.899, 'pos': 0.036}


